In [1]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd

In [2]:
ROUTES_PATH = 'http://tosamara.ru/api/classifiers/routesAndStopsCorrespondence.xml'

In [13]:
req = requests.get(ROUTES_PATH)
tree = ET.fromstring(req.text)
tree

<Element 'routes' at 0x0000025417EAF548>

In [46]:
routes = []
for child in tree:
    number = child[1].text
    direction = child[2].text
    type_id = int(child[3][0].text)
    performing = bool(child[4].text)
    geometry = child[-1].text.split(' ')
    routes.append({'number': number,
                  'direction': direction,
                  'type_id': type_id,
                  'performing' : performing,
                  'geometry' : geometry})

In [47]:
df_routes = pd.DataFrame.from_records(routes)
df_routes.head()

,number,direction,type_id,performing,geometry
0,1,Железнодорожный вокзал,1,True,"[53.384692,50.169369, 53.384749,50.169603, 53...."
1,1,Красная Глинка,1,True,"[53.187111,50.121445, 53.18814,50.122168, 53.1..."
2,21,Юнгородок,1,True,"[53.259224,50.216663, 53.259209,50.216672, 53...."
3,21,Барбошина Поляна,1,True,"[53.213223,50.291229, 53.213392,50.291594, 53...."
4,27,"кинотеатр ""Луч""",1,True,"[53.213391,50.247921, 53.217799,50.24232, 53.2..."


In [103]:
len(df_routes.number.unique())

169

In [63]:
def parse_geom(x):
    points = [(float(i.split(',')[0]),float(i.split(',')[1])) for i in x]
    edges = [(points[i-1],points[i]) for i in range(1, len(points))]
    return edges
df_routes['geo'] = df_routes['geometry'].apply(parse_geom)

In [64]:
global_edges = set()
for i in df_routes['geo'].values:
    global_edges = global_edges | set(i)

In [65]:
len(global_edges)

13309

In [66]:
df_routes.head()

,number,direction,type_id,performing,geometry,geo
0,1,Железнодорожный вокзал,1,True,"[53.384692,50.169369, 53.384749,50.169603, 53....","[((53.384692, 50.169369), (53.384749, 50.16960..."
1,1,Красная Глинка,1,True,"[53.187111,50.121445, 53.18814,50.122168, 53.1...","[((53.187111, 50.121445), (53.18814, 50.122168..."
2,21,Юнгородок,1,True,"[53.259224,50.216663, 53.259209,50.216672, 53....","[((53.259224, 50.216663), (53.259209, 50.21667..."
3,21,Барбошина Поляна,1,True,"[53.213223,50.291229, 53.213392,50.291594, 53....","[((53.213223, 50.291229), (53.213392, 50.29159..."
4,27,"кинотеатр ""Луч""",1,True,"[53.213391,50.247921, 53.217799,50.24232, 53.2...","[((53.213391, 50.247921), (53.217799, 50.24232..."


In [67]:
req = requests.get('http://www.samaratrans.info/wiki/index.php/%D0%A1%D0%B0%D0%BC%D0%B0%D1%80%D0%B0_%D0%B0%D0%B2%D1%82%D0%BE%D0%B1%D1%83%D1%81_2')
html_text = req.text

In [68]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_text, 'html.parser')

In [69]:
collect_links = []
for j in soup.find_all('ul')[-6:]:
    for i in j.find_all('a'):
    try:
        if "маршруты" not in i['title'] and "расписания" not in i['title']:
        collect_links.append(i['href'])
    except KeyError:
        pass
len(collect_links)

199

In [70]:
def parse_add_inf(t):
    additional_information = t.find_all('td')
    add_inf = {}
    for i in range(0,14,2):
        add_inf[additional_information[i].text.strip()] = additional_information[i+1].text.strip()
    return add_inf


def parse_route_page(soup):
    try:
        tables = soup.find_all('table')
        route_name = tables[2].find_all('li')[0].text.split('\n\n')[0]
        return route_name, parse_add_inf(tables[7])
    except:
        return None, None

In [72]:
DOMAIN_LINK = 'http://www.samaratrans.info'

parsed_routes = {}

# for link in collect_links:
#     r = requests.get(DOMAIN_LINK+link)
#     s = BeautifulSoup(r.text, 'html.parser')
#     name, inf = parse_route_page(s)
#     parsed_routes[name] = inf
  

In [3]:
# with open('file.txt', 'w') as f:
#     f.write(str(parsed_routes))
with open('file.txt', 'r') as f:
    parsed_routes = eval(f.read())
parsed_routes

{'1 Автобусный маршрут № 1 "а/ст "Красная Глинка" - Ж/д вокзал (Поликлиника)"': {'Режим работы': 'Ежедневно: с 6 до 22 часов',
  'Интервалы движения': 'По рабочим дням: 8-20 мин.По выходным дням: 16-35 мин.В вечернее время (после 19 часов): 20-60 мин.',
  'Выпуск на линию': 'По рабочим дням: 20 машинПо выходным дням: 8 машин',
  'Время в пути': '1 ч. 28 мин.',
  'Протяжённость маршрута': '32,2 км',
  'Обслуживающие предприятия': 'ООО "СамараАвтоГаз"',
  'Подвижной состав': 'ЛиАЗ-5293.70'},
 '1 Автобусный маршрут № 3 "Хлебная площадь - п. Шмидта (Речная)"': {'Режим работы': 'Ежедневно: с 7 до 20 часов',
  'Интервалы движения': '48-60 мин.',
  'Выпуск на линию': '1 машина',
  'Время в пути': '28 мин.',
  'Протяжённость маршрута': '7,0 км',
  'Обслуживающие предприятия': 'ООО "СамараАвтоГаз"',
  'Подвижной состав': 'МАЗ-206'},
 '1 Автобусный маршрут № 4 "ул. Олимпийская - Ж/д вокзал"': {'Режим работы': 'Ежедневно: с 6 до 21 часа',
  'Интервалы движения': '7-10 мин.',
  'Выпуск на линию': 

In [4]:
type_dict = {int(i.split(' — ')[0]): i.split(' — ')[1] for i in '''1 — автобус, 2 — метрополитен, 3 — трамвай, 4 — троллейбус, 5 — электропоезд, 6 — речной транспорт'''.split(', ')}
type_dict

{1: 'автобус',
 2: 'метрополитен',
 3: 'трамвай',
 4: 'троллейбус',
 5: 'электропоезд',
 6: 'речной транспорт'}

In [211]:
routes_inf = []
exceptions = {}
for key, value in parsed_routes.items():
    try:
        key_splited = key.split(' ')
        number = key_splited[4]
        type_title = key_splited[1]
        schedule_days = { i.split(':')[0]: i.split(':')[1] 
                 for i in value['Режим работы'].split('По') if len(i.split(':')) > 1}
        intervals = value['Интервалы движения']
        vehicles = value['Подвижной состав']
        routes_inf.append([number, type_title, schedule_days, intervals, vehicles])
    except:
        print(key, value)
        exceptions[key] = value

1 Автобусный маршрут № 7 "Безымянский рынок - ул. Транзитная (кольцевой)" {'отправление от Безымянского рынка': 'часы', '05': '06', '07': '08', '09': '10', '11': '12', '13': '14', '15': '16'}
1 Автобусный маршрут № 27 "Безымянский рынок - Кинотеатр "Луч" (кольцевой)" {'отправление от Безымянского рынка': 'часы', '05': '06', '07': '08', '09': '10', '11': '12', '13': '14', '15': '16'}
1 Автобусный маршрут № 59 "15-й микрорайон (ТЦ "Колизей") - Пост ГИБДД (кольцевой)" {'отправление от ТЦ "Колизей"': 'часы', '05': '06', '07': '08', '09': '10', '11': '12', '13': '14', '15': '16'}
1 Автобусный маршрут № 59а "15-й микрорайон (ТЦ "Колизей") - Пост ГИБДД (кольцевой)" {'отправление от ТЦ "Колизей"': 'часы', '05': '06', '07': '08', '09': '10', '11': '12', '13': '14', '15': '16'}
None None
1 Автобусный маршрут № 126б "Завод "Металлург" - СДТ "Советы"" {'Дни работы': 'С 1 мая по 31 октября: по выходным и праздничным днямС 1 мая по 30 сентября: дополнительно по средам и пятницам(кроме дней массовых 

In [212]:
for key, value in exceptions.items():
    try:
        key_splited = key.split(' ')
        number = key_splited[4]
        type_title = key_splited[1]
        schedule_days = {value['Дни работы'] : value['Время работы'] }
        intervals = value['Интервалы движения']
        vehicles = value['Выпуск на линию']
        routes_inf.append([number, type_title, schedule_days, intervals, vehicles])
    except:
        print(key, value)
        exceptions[key] = value

1 Автобусный маршрут № 7 "Безымянский рынок - ул. Транзитная (кольцевой)" {'отправление от Безымянского рынка': 'часы', '05': '06', '07': '08', '09': '10', '11': '12', '13': '14', '15': '16'}
1 Автобусный маршрут № 27 "Безымянский рынок - Кинотеатр "Луч" (кольцевой)" {'отправление от Безымянского рынка': 'часы', '05': '06', '07': '08', '09': '10', '11': '12', '13': '14', '15': '16'}
1 Автобусный маршрут № 59 "15-й микрорайон (ТЦ "Колизей") - Пост ГИБДД (кольцевой)" {'отправление от ТЦ "Колизей"': 'часы', '05': '06', '07': '08', '09': '10', '11': '12', '13': '14', '15': '16'}
1 Автобусный маршрут № 59а "15-й микрорайон (ТЦ "Колизей") - Пост ГИБДД (кольцевой)" {'отправление от ТЦ "Колизей"': 'часы', '05': '06', '07': '08', '09': '10', '11': '12', '13': '14', '15': '16'}
None None
1 Автобусный маршрут № 157 "Площадь им. Кирова - Чёрновские дачи" {'отправление от площади им. Кирова': 'часы', '05': '06', '07': '08', '09': '10', '11': '12', '13': '14', '15': '16'}
1 Автобусный маршрут № 174 

In [213]:
routes_inf.append(['7', 'Автобусный', {'Ежедневно': ' с 6 до 20 часов'}, 'По рабочим дням: 12-84 мин.По выходным дням: 68-164 мин.', 'ПАЗ-3203ПАЗ-3204ПАЗ-32054'])
routes_inf.append(['27', 'Автобусный', {'Ежедневно': ' с 6 до 20 часов'}, 'По рабочим дням: 12-44 мин.По субботам: 14-46 мин.По воскресеньям: 18-58 мин.', 'Hyundai County Kuzbas HDU2ПАЗ-3203ПАЗ-3204ПАЗ-32054'])
routes_inf.append(['59', 'Автобусный', {'Ежедневно': ' с 6 до 19 часов'}, '77 мин.', 'ПАЗ-3203'])
routes_inf.append(['59а', 'Автобусный', {'Ежедневно': ' с 6 до 19 часов'}, '77 мин.', 'ПАЗ-3203'])
routes_inf.append(['157', 'Автобусный', {'С 1 мая по 31 октября: по выходным и праздничным днямС 1 мая по 30 сентября: дополнительно по средам и пятницам(кроме дней массовых посещений кладбищ)': 'С 7 до 12 часов и с 15 до 20 часов'}, 'По рабочим дням: 30-35 мин.По выходным дням: 25-50 мин.', 'ЛиАЗ-5293.70МАЗ-206'])
routes_inf.append(['174', 'Автобусный', {'С 1 мая по 31 октября: по выходным и праздничным днямС 1 мая по 30 сентября: дополнительно по средам и пятницам(кроме дней массовых посещений кладбищ)': 'С 7 до 12 часов и с 16 до 19 часов'}, '45-50 мин.', 'МАЗ-206'])
routes_inf.append(['4', 'Трамвайный', {'Ежедневно': 'с 5 до 21 часа'}, '14 мин.', 'Tatra T3SU (Т-3) (одиночные вагоны)АКСМ-62103 (одиночные вагоны)'])
routes_inf.append(['4', 'Трамвайный', {' рабочим дням': ' с 6 до 23 часов', ' выходным дням': ' с 6 до 22 часов'}, 'По рабочим дням: 16 мин.По выходным дням: 14 мин.', 'Tatra T3SU (Т-3) (одиночные вагоны)АКСМ-62103 (одиночные вагоны)71-402 "СПЕКТР" (одиночные вагоны)'])

In [214]:
df_routes_inf = pd.DataFrame(routes_inf, columns=['number', 'type_title', 'schedule_days', 'intervals', 'vehicles'])
df_routes_inf

,number,type_title,schedule_days,intervals,vehicles
0,1,Автобусный,{'Ежедневно': ' с 6 до 22 часов'},По рабочим дням: 8-20 мин.По выходным дням: 16...,ЛиАЗ-5293.70
1,3,Автобусный,{'Ежедневно': ' с 7 до 20 часов'},48-60 мин.,МАЗ-206
2,4,Автобусный,{'Ежедневно': ' с 6 до 21 часа'},7-10 мин.,Автобусы малого класса
3,5д,Автобусный,{'Ежедневно': ' с 5 до 22 часов'},По рабочим дням: 10-20 мин.По выходным дням: 1...,ЛиАЗ-5293.70
4,6,Автобусный,{'Ежедневно': ' с 6 до 21 часа'},По рабочим дням: 15-34 мин.По выходным дням: 1...,МАЗ-206
5,8,Автобусный,{'Ежедневно': ' с 6 до 13 часов и с 15 до 20 ч...,60 мин.,ПАЗ-3203ПАЗ-3204
6,9,Автобусный,{'Ежедневно': ' с 6 до 22 часов'},По рабочим дням: 18-40 мин.По субботам: 16-40 ...,МАЗ-206
7,11,Автобусный,{'Ежедневно': ' с 6 до 19 часов'},94-100 мин.,ПАЗ-3203ПАЗ-3204ПАЗ-32054
8,12,Автобусный,{' рабочим дням': ' с 6 до 8 часов и с 14 до 1...,48 мин.,ПАЗ-3203ПАЗ-3204ПАЗ-32054
9,13,Автобусный,{'Ежедневно': ' с 7 до 20 часов'},58-60 мин.,МАЗ-206


In [215]:
df_routes_inf.iloc[10].schedule_days = {'Ежедневно': 'С 7 до 20 часов'}
df_routes_inf.iloc[26].schedule_days = {'Ежедневно': 'С 6 до 20 часов'}

In [216]:
set([j for i in df_routes_inf.schedule_days.values for j in i.keys()])

{' выходным дням',
 ' рабочим дням',
 'В дни проведения мероприятий на стадионе "Самара Арена"Начало движения',
 'Выходные дни',
 'Ежедневно',
 'Пасха, Красная горка, Радоница, Троицкая родительская суббота, День Победы',
 'С 1 мая по 31 октября: по выходным и праздничным дням(кроме дней массовых посещений кладбищ)',
 'С 1 мая по 31 октября: по выходным и праздничным днямС 1 мая по 30 сентября: дополнительно по вторникам и четвергам(кроме дней массовых посещений кладбищ)',
 'С 1 мая по 31 октября: по выходным и праздничным днямС 1 мая по 30 сентября: дополнительно по средам и пятницам(кроме дней массовых посещений кладбищ)'}

Пасха, Красная горка, Радоница, Троицкая родительская суббота, День Победы 

Пасха - всегда воскресение
Красная горка- через неделю после Пасхи, тоже воскресение
Радоница - 9 дней после пасхи - вторник
Троицкая родительская суббота - суббота
День Победы - всегда выходной

In [217]:
def parse_time(text_time):
    words = text_time.strip().split(' ')
    if 'и ' in text_time:
        return list(range(int(words[1]), int(words[3]))) + list(range(int(words[7]), int(words[9])))
    return list(range(int(words[1]), int(words[3])))

def schedule_day(x):
    keys = x.schedule_days.keys()
    work_day, saturday, sunday, religion, arena, summer_weekend, summer_thu_tue, summer_wed_fri = [None for i in range(8)]
    if 'Ежедневно' in keys:
        work_day, saturday, sunday = [parse_time(x.schedule_days['Ежедневно']) for i in range(3)]
    if ' выходным дням' in keys:
        saturday, sunday = [parse_time(x.schedule_days[' выходным дням']) for i in range(2)]
    if 'Выходные дни' in keys:
        saturday, sunday = [parse_time(x.schedule_days['Выходные дни']) for i in range(2)]
    if ' рабочим дням' in keys:
        work_day = parse_time(x.schedule_days[' рабочим дням'])
    if 'В дни проведения мероприятий на стадионе "Самара Арена"Начало движения' in keys:
        arena = parse_time('с 15 до 23 часов')
    if 'Пасха, Красная горка, Радоница, Троицкая родительская суббота, День Победы' in keys:
        religion = parse_time(x.schedule_days['Пасха, Красная горка, Радоница, Троицкая родительская суббота, День Победы'])
    if 'С 1 мая по 31 октября: по выходным и праздничным дням(кроме дней массовых посещений кладбищ)' in keys:
        summer_weekend = parse_time(x.schedule_days['С 1 мая по 31 октября: по выходным и праздничным дням(кроме дней массовых посещений кладбищ)'])
    if 'С 1 мая по 31 октября: по выходным и праздничным днямС 1 мая по 30 сентября: дополнительно по вторникам и четвергам(кроме дней массовых посещений кладбищ)' in keys:
        summer_thu_tue = parse_time(x.schedule_days['С 1 мая по 31 октября: по выходным и праздничным днямС 1 мая по 30 сентября: дополнительно по вторникам и четвергам(кроме дней массовых посещений кладбищ)'])
    if 'С 1 мая по 31 октября: по выходным и праздничным днямС 1 мая по 30 сентября: дополнительно по средам и пятницам(кроме дней массовых посещений кладбищ)' in keys:
        summer_wed_fri = parse_time(x.schedule_days['С 1 мая по 31 октября: по выходным и праздничным днямС 1 мая по 30 сентября: дополнительно по средам и пятницам(кроме дней массовых посещений кладбищ)'])
    return pd.Series([work_day, saturday, sunday, religion, arena, summer_weekend, summer_thu_tue, summer_wed_fri])

df_routes_inf[['work_day', 'saturday', 'sunday', 'religion', 'arena', 'summer_weekend', 'summer_thu_tue', 'summer_wed_fri']] = df_routes_inf.apply(schedule_day, axis=1)

In [218]:
df_routes_inf.head()

,number,type_title,schedule_days,intervals,vehicles,work_day,saturday,sunday,religion,arena,summer_weekend,summer_thu_tue,summer_wed_fri
0,1,Автобусный,{'Ежедневно': ' с 6 до 22 часов'},По рабочим дням: 8-20 мин.По выходным дням: 16...,ЛиАЗ-5293.70,"[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...","[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...","[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...",None,None,None,None,None
1,3,Автобусный,{'Ежедневно': ' с 7 до 20 часов'},48-60 мин.,МАЗ-206,"[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]","[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]","[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]",None,None,None,None,None
2,4,Автобусный,{'Ежедневно': ' с 6 до 21 часа'},7-10 мин.,Автобусы малого класса,"[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...","[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...","[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...",None,None,None,None,None
3,5д,Автобусный,{'Ежедневно': ' с 5 до 22 часов'},По рабочим дням: 10-20 мин.По выходным дням: 1...,ЛиАЗ-5293.70,"[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17...","[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17...","[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17...",None,None,None,None,None
4,6,Автобусный,{'Ежедневно': ' с 6 до 21 часа'},По рабочим дням: 15-34 мин.По выходным дням: 1...,МАЗ-206,"[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...","[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...","[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...",None,None,None,None,None


In [220]:
df_routes_inf[df_routes_inf.religion.notna()]

,number,type_title,schedule_days,intervals,vehicles,work_day,saturday,sunday,religion,arena,summer_weekend,summer_thu_tue,summer_wed_fri
171,4к,Автобусный,"{'Пасха, Красная горка, Радоница, Троицкая род...",20-40 мин.,1-2 машины,None,None,None,"[7, 8, 9, 10, 11, 12, 13, 14]",None,None,None,None
172,27к,Автобусный,"{'Пасха, Красная горка, Радоница, Троицкая род...",10-20 мин.,4-6 машин,None,None,None,"[7, 8, 9, 10, 11, 12, 13, 14]",None,None,None,None
173,36д,Автобусный,"{'Пасха, Красная горка, Радоница, Троицкая род...",20-30 мин.,3-4 машины,None,None,None,"[7, 8, 9, 10, 11, 12, 13, 14]",None,None,None,None
174,36ю,Автобусный,"{'Пасха, Красная горка, Радоница, Троицкая род...",30 мин.,2 машины,None,None,None,"[8, 9, 10, 11, 12, 13]",None,None,None,None
175,63э,Автобусный,"{'Пасха, Красная горка, Радоница, Троицкая род...",5-10 мин.,До 25 машин,None,None,None,"[7, 8, 9, 10, 11, 12, 13, 14]",None,None,None,None
176,67к,Автобусный,"{'Пасха, Красная горка, Радоница, Троицкая род...",30-60 мин.,1-2 машины,None,None,None,"[7, 8, 9, 10, 11, 12, 13, 14]",None,None,None,None
177,77д,Автобусный,"{'Пасха, Красная горка, Радоница, Троицкая род...",15-20 мин.,4-5 машин,None,None,None,"[7, 8, 9, 10, 11, 12, 13, 14]",None,None,None,None
178,129э,Автобусный,"{'Пасха, Красная горка, Радоница, Троицкая род...",5-10 мин.,До 20 машин,None,None,None,"[7, 8, 9, 10, 11, 12, 13, 14]",None,None,None,None


In [221]:
df_routes_inf.intervals = df_routes_inf.intervals.apply(lambda x: '360 мин' if x == '-' else x)
df_routes_inf.intervals = df_routes_inf.intervals.apply(lambda x:  x.replace('до ', '') if 'до 80 мин.' in x else x)

In [222]:
def parse_mints(mints):
    if '-' in mints:
        mints = [int(i.strip()) for i in mints.split('-')]
        return sum(mints)/len(mints)
    return int(mints.strip())

def parse_intervals(x):
    parts = x.split('.')
    try:
        if ':' in x:
            return {i.split(':')[0] : parse_mints(i.split(':')[1].replace(' мин', '')) for i in parts if ':' in i}
        return {'always': parse_mints(parts[0].replace(' мин', ''))}
    except:
        print(x)
df_routes_inf['intervals_dict'] = df_routes_inf.intervals.apply(parse_intervals)
# t = 'По рабочим дням: 10-20 мин.По выходным дням: 20-30 мин.В вечернее время (после 20 часов): 30 мин.'
# parse_intervals(t)

In [226]:
# set([j for i in df_routes_inf.schedule_days.values for j in i.values()])
# df_routes_inf.intervals.unique()
# df_routes_inf.head()

In [224]:
def schedule_intervals(x):
    keys = x.intervals_dict.keys()
    work_day_int, sunday_int, saturday_int, int_18, int_19, int_20 = [None for i in range(6)]  
    if 'always' in keys:
        work_day_int, sunday_int, saturday_int, int_18, int_19, int_20 = [x.intervals_dict['always'] for i in range(6)]
    if 'По рабочим дням' in keys:
        work_day_int = x.intervals_dict['По рабочим дням'] 
    if 'По выходным дням' in keys:
        sunday_int, saturday_int = x.intervals_dict['По выходным дням'], x.intervals_dict['По выходным дням'] 
    if 'По субботам' in keys:
        saturday_int = x.intervals_dict['По субботам']
    if 'По воскресеньям' in keys:
        sunday_int = x.intervals_dict['По воскресеньям']
    if 'В вечернее время (после 18 часов)' in keys:
        int_18, int_19, int_20 = [x.intervals_dict['В вечернее время (после 18 часов)'] for i in range(3)]
    if 'В вечернее время (после 19 часов)' in keys:
        int_19, int_20 = [x.intervals_dict['В вечернее время (после 19 часов)'] for i in range(2)]
    if 'В вечернее время (после 20 часов)' in keys:
        int_20 = x.intervals_dict['В вечернее время (после 20 часов)']
    return pd.Series([work_day_int, sunday_int, saturday_int, int_18, int_19, int_20])

df_routes_inf[['work_day_int', 'sunday_int', 'saturday_int', 'int_18', 'int_19', 'int_20']] = df_routes_inf.apply(schedule_intervals, axis=1)

In [225]:
df_routes_inf.head()

,number,type_title,schedule_days,intervals,vehicles,work_day,saturday,sunday,religion,arena,summer_weekend,summer_thu_tue,summer_wed_fri,intervals_dict,work_day_int,sunday_int,saturday_int,int_18,int_19,int_20
0,1,Автобусный,{'Ежедневно': ' с 6 до 22 часов'},По рабочим дням: 8-20 мин.По выходным дням: 16...,ЛиАЗ-5293.70,"[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...","[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...","[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...",None,None,None,None,None,"{'По рабочим дням': 14.0, 'По выходным дням': ...",14.0,25.5,25.5,NaN,40.0,40.0
1,3,Автобусный,{'Ежедневно': ' с 7 до 20 часов'},48-60 мин.,МАЗ-206,"[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]","[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]","[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]",None,None,None,None,None,{'always': 54.0},54.0,54.0,54.0,54.0,54.0,54.0
2,4,Автобусный,{'Ежедневно': ' с 6 до 21 часа'},7-10 мин.,Автобусы малого класса,"[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...","[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...","[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...",None,None,None,None,None,{'always': 8.5},8.5,8.5,8.5,8.5,8.5,8.5
3,5д,Автобусный,{'Ежедневно': ' с 5 до 22 часов'},По рабочим дням: 10-20 мин.По выходным дням: 1...,ЛиАЗ-5293.70,"[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17...","[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17...","[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17...",None,None,None,None,None,"{'По рабочим дням': 15.0, 'По выходным дням': ...",15.0,19.5,19.5,25.5,25.5,25.5
4,6,Автобусный,{'Ежедневно': ' с 6 до 21 часа'},По рабочим дням: 15-34 мин.По выходным дням: 1...,МАЗ-206,"[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...","[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...","[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1...",None,None,None,None,None,"{'По рабочим дням': 24.5, 'По выходным дням': ...",24.5,48.0,48.0,NaN,NaN,NaN


In [230]:
for i in df_routes_inf.vehicles.unique():
    print(i)


ЛиАЗ-5293.70
МАЗ-206
Автобусы малого класса
ПАЗ-3203ПАЗ-3204
ПАЗ-3203ПАЗ-3204ПАЗ-32054
ЛиАЗ-5293.70МАЗ-206
Hyundai County Kuzbas HDU2
ПАЗ-3205
Hyundai County Kuzbas HDU2Hyundai County LWBПАЗ-3205ПАЗ-4234
ПАЗ-3203ПАЗ-3204ПАЗ-32054ПАЗ-4234МАЗ-206 (рабочие дни)
Hyundai County Kuzbas HDU2Богдан А201.11ПАЗ-3203
Hyundai County Kuzbas HDU2ПАЗ-3203ПАЗ-3204ПАЗ-32054
Hyundai County Kuzbas HDU2ПАЗ-3203ПАЗ-3204
ПАЗ-3205, ПАЗ-4234
Автобусы малого класса, Богдан А092, ПАЗ-3204
ПАЗ-3205Богдан А092
Tatra T3SU (Т-3) (одиночные вагоны)АКСМ-62103 (одиночные вагоны)71-623-02 (одиночные вагоны)
Tatra T3SU (Т-3) (одиночные вагоны)
Tatra T3SU (Т-3) (двухвагонные поезда и одиночные вагоны)АКСМ-62103 (одиночные вагоны)
Tatra T3SU (Т-3) (двухвагонные поезда)71-623-02 (двухвагонные поезда)71-631 (трёхсекционные вагоны)
Tatra T6B5 (Т-3М) (двухвагонные поезда)Tatra T3SU (Т-3) (двухвагонные поезда)71-405 (двухвагонные поезда)71-631 (трёхсекционные вагоны)71-623-02 (двухвагонные поезда)АКСМ-62103 (одиночные вагоны)
